This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Fit a plane to selected projection centers

In this tutorial, we will fit a plane to projection centers (PCs) determined from a grid of (5, 5) EBSD patterns from a single crystal silicon wafer.
The resulting fitted plane contains one PC per pattern in the dataset from which the grid is obtained.
The plane of PCs can be used in subsequent indexing of the dataset.
To relate the sample positions of the grid to the PCs, we will test both an affine transformation and a projective transformation, following the work of <cite data-cite="winkelmann2020refined">Winkelmann et al. (2020)</cite>.

We'll start by importing necessary libraries

In [ ]:
%matplotlib inline

from diffsims.crystallography import ReciprocalLatticeVector
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np
from orix.crystal_map import CrystalMap, create_coordinate_arrays, PhaseList
from orix.quaternion import Rotation
from pyebsdindex import ebsd_index, pcopt
import skimage.filters as skf


plt.rcParams.update(
    {"figure.facecolor": "w", "font.size": 12, "figure.dpi": 125}
)

## Load and inspect data

Load data lazily (~500 MB) and inspect its shape and step size

In [ ]:
s = kp.data.si_wafer(allow_download=True, lazy=True)
s

We see that we have (50, 50) patterns of shape (480, 480), which is the full pattern resolution of the NORDIF UF-420 detector the patterns are acquired on.
In the axes manager

In [ ]:
print(s.axes_manager)

... we see that the nominal step sizes are $(\Delta x, \Delta y) = (40, 40)$ $\mu$m.
The scan spans an area of $(2 \times 2)$ $\mu$m$^2$.
The NORDIF UF-420 detector pixel size (unbinned) is about 90 $\mu$m.
This means that, for example, we expect the PC to shift about 2000 / 90 $\approx$ 22 detector pixels when moving horizontally in the scan.
To get a better understanding of the expected changes of the PC in the scan, we'll look at the mean intensity map of the sample.

In [ ]:
s_mean_nav = s.mean(axis=(2, 3))
s_mean_nav.compute()  # Need this call because data was loaded lazily

Plot it, also annotating the orientation of the scan relative to the sample position in the chamber.
We use the [plot()](https://orix.readthedocs.io/en/stable/reference/generated/orix.crystal_map.CrystalMap.plot.html) method of the [EBSD.xmap](../reference/generated/kikuchipy.signals.EBSD.xmap.rst) attribute for this.

In [ ]:
s.xmap.scan_unit = "um"
fig = s.xmap.plot(
    s_mean_nav.data.ravel(),
    colorbar=True,
    colorbar_label="Mean intensity",
    return_figure=True,
)

# Annotate
ax = fig.axes[0]
ax.text(25, 1, "Bottom", va="top", ha="center")
ax.text(25, 49, "Top", va="bottom", ha="center");

The brighter and darker spots come from patterns acquired close to fiducial markers on the Si wafer, while the vertical gradient is a result of the Gaussian intensity distribution varying across the sample.
The orientation of the EBSD scan is important to note: To avoid depositing carbon ahead of the scanning beam, the sample is scanned from down the sample upwards, annotated in the map above ("Bottom", "Top").

The Bruker PC convention is used internally in kikuchipy.
The convention measures (PCx, PCy, PCz) in the following manner, viewing the detector from behind the detector towards the sample:
* PCx from left towards the right
* PCy downwards from the top
* PCz as the shortest distance from the beam-sample interaction position to the detector in fractions of the detector width.

See the [reference frames tutorial](reference_frames.ipynb) for more details and conversions between conventions.

Given this definition, let's annotate on the above map the expected changes to the PC (increasing in the direction of the arrow)

In [ ]:
annotate_kw = dict(
    arrowprops=dict(arrowstyle="-|>", lw=2, mutation_scale=15)
)
xys = [(35, 25), (40, 35), (40, 15)]
xy_texts = [(49, 25), (40, 49), (40, 1)]
has = ["right", "center", "center"]
vas = ["center", "bottom", "top"]
labels = ["PCx", "PCy", "PCz"]
for xy, xytext, ha, va, label in zip(xys, xy_texts, has, vas, labels):
    ax.annotate(label, xy=xy, xytext=xytext, ha=ha, va=va, **annotate_kw)

# Show figure again with new annotations
fig

Our goal is to fit a plane of (PCx, PCy, PCz) to this (50, 50) map with these variations.
To this end, we will perform the following actions:
1. Extract an evenly spaced grid of patterns from the full dataset
2. Get initial guesses of the PC for each grid pattern with Hough indexing (from PyEBSDIndex)
3. Get initial guesses of the orientation for each grid pattern using the PCs with Hough indexing
4. Refine the orientations and PCs simultaneously by matching experimental to dynamically simulated patterns (simulated with EMsoft)
5. Fit a plane to the refined PCs

## Extract a grid of patterns

Before we extract the grid of patterns and prepare these for indexing by background correction, we obtain a binary mask to remove parts of the pattern without information (typically the corners).
We make the mask via thresholding, and find the threshold value by using the minimum threshold algorithm implemented in scikit-image; note that they have other thresholding algorithms that might work better for other datasets.

In [ ]:
s_mean = s.mean((0, 1))  # (480, 480)
mean_data = s_mean.data.compute()

In [ ]:
# Threshold
threshold = skf.threshold_minimum(mean_data)
mask = mean_data < threshold

# Extract center pattern for plotting
p0 = s.inav[25, 25].data

fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
for ax, arr, title in zip(
    axes,
    [p0, mask, p0 * ~mask],
    ["Pattern", "Mask", "Pattern * ~mask"]
):
    ax.imshow(arr, cmap="gray")
    ax.axis("off")
    ax.set_title(title)
fig.subplots_adjust(wspace=0)

The mask efficiently removes the parts of the pattern which contains no information of interest.

Extract the grid using [EBSD.extract_grid()](../reference/generated/kikuchipy.signals.EBSD.extract_grid.rst)

In [ ]:
grid_shape = (5, 5)
s_grid, idx = s.extract_grid(grid_shape, return_indices=True)

nav_shape_grid = s_grid.axes_manager.navigation_shape[::-1]
sig_shape_grid = s_grid.axes_manager.signal_shape[::-1]

s_grid.compute()

s_grid

Let's plot the grid positions on the navigation map

In [ ]:
kp.draw.plot_pattern_positions_in_map(
    rc=idx.reshape(2, -1).T,
    roi_shape=(50, 50),
    roi_image=s_mean_nav.data,
)

## Initial guess of PC using Hough indexing

Prepare grid patterns for PC optimization and Hough indexing with PyEBSDIndex by background removal and setting the masked out values to 0

In [ ]:
s_grid.remove_static_background()
s_grid.remove_dynamic_background()

patterns_grid = s_grid.data.reshape((-1,) + sig_shape_grid).copy()
patterns_grid[:, mask] = 0

Create indexer

In [ ]:
indexer = ebsd_index.EBSDIndexer(
    phaselist=["FCC"],
    vendor="kikuchipy",
    sampleTilt=70,
    camElev=0,
    patDim=sig_shape_grid,
    nBands=6,  # Default is 9
)

Optimize PCs of the patterns in the grid using an initial guess based on previous knowledge of PCs in the same microscope

In [ ]:
pc_grid = pcopt.optimize(
    pats=patterns_grid, indexer=indexer, PC0=[0.52, 0.16, 0.50], batch=True
)

print(pc_grid.mean(0))
print(pc_grid.std(0))

Store the PCs in an [EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst) and plot them

In [ ]:
det_grid = kp.detectors.EBSDDetector(
    shape=sig_shape_grid,
    pc=pc_grid.reshape(nav_shape_grid + (3,)),
    sample_tilt=70,
)
det_grid

In [ ]:
det_grid.plot_pc("map")
det_grid.plot_pc("scatter", annotate=True)

Looking at the maps:
* PCx increases towards the left, as expected.
* PCy increases upwards, as expected.
* PCz seems to increase downwards, as expected, although there are some patterns, especially the 0th, which do not follow this trend.

Looking at the scatter plots:
* We recognize the regular (5, 5) grid, although some patterns do not align as well as the others (e.g. 0th).
* There is an inverse relation between PCy and PCz: as PCy decreases, PCz increases. This is as expected. However, there is a significant spread in each row of points (5-9, 10-14, etc.).

These initial guesses seem OK, and we could perhaps fit a plane to these values already.
But, we can get a better fit by refining the PC values using pattern matching.
To that end, we obtain initial guesses for the orientations

In [ ]:
plt.figure()
hi_grid, *_ = indexer.index_pats(
    patsin=patterns_grid, PC=det_grid.pc_flattened, verbose=2
)

Create a [CrystalMap](https://orix.readthedocs.io/en/stable/reference/generated/orix.crystal_map.CrystalMap.html#orix.crystal_map.CrystalMap) to store the indexing results in.
We need a description of the Si phase when creating the crystal map.
To obtain this, we load the dynamically simulated (with EMsoft) Si master pattern which we will use below for pattern matching.

In [ ]:
mp = kp.data.si_ebsd_master_pattern(
    allow_download=True,
    projection="lambert",
    energy=20,
)
mp

Extract the phase and change lattice constant unit from nm to Ångström

In [ ]:
phase = mp.phase

lat = phase.structure.lattice
lat.setLatPar(lat.a * 10, lat.b * 10, lat.c * 10)

print(phase)
print(phase.structure)
print(phase.structure.lattice)

Create coordinate arrays and the crystal map

In [ ]:
xy_grid, _ = create_coordinate_arrays(
    nav_shape_grid,
    step_sizes=(
        s_grid.axes_manager["y"].scale,
        s_grid.axes_manager["x"].scale,
    ),
)

hi_grid_best = hi_grid[-1]

xmap_grid = CrystalMap(
    rotations=Rotation(hi_grid_best["quat"]),
    x=xy_grid["x"].ravel(),
    y=xy_grid["y"].ravel(),
    phase_list=PhaseList(phase),
    prop=dict(
        pq=hi_grid_best["pq"],
        cm=hi_grid_best["cm"],
        fit=hi_grid_best["fit"],
        nmatch=hi_grid_best["nmatch"],
    ),
    scan_unit="um",
)
xmap_grid

Check the average pattern fit and confidence metric

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10, 3.5))
for ax, to_plot, label in zip(
    axes,
    ["fit", "cm"],
    ["Pattern fit [deg]", "Confidence metric"]
):
    im = ax.imshow(xmap_grid.get_map_data(to_plot))
    fig.colorbar(im, ax=ax, label=label)
    ax.axis("off")
fig.subplots_adjust(wspace=0.1)

We see that the upper left (0th) pattern has a high pattern fit and low confidence metric, which is in line with the PC values of that pattern being an outlier in the scatter plots of (PCx, PCz) and (PCz, PCy) above.

Let us refine these initial guesses of orientations and PCs using the loaded dynamically simulated master pattern.

## Refine PCs using pattern matching

Let's look at the dynamical simulation (which is in the square Lambert projection, required for use in dictionary indexing and refinement in kiuchipy [and EMsoft])

In [ ]:
mp.plot()

Refine orientations and PCs using the Nelder-Mead optimization algorithm as implemented in NLopt.
During optimization, orientations are represented as three Euler angles, and are varied within a sufficiently wide trust region of +/- 5$^{\circ}$ of the orientation obtained from Hough indexing above.
PCs are varied within a trust region of +/- 0.1 of the detector width (for PCx and PCz) or height (for PCy).
Optimization of each pattern stops when the increase in normalized cross-correlation (NCC) score between iterations, obtained by comparing experimental to simulated pattern, is lower than $10^{-5}$.

In [ ]:
xmap_grid_ref, det_grid_ref = s_grid.refine_orientation_projection_center(
    xmap=xmap_grid,
    detector=det_grid,
    master_pattern=mp,
    energy=20,
    signal_mask=mask,
    method="LN_NELDERMEAD",
    trust_region=[5, 5, 5, 0.1, 0.1, 0.1],
    rtol=1e-5,
    chunk_kwargs=dict(chunk_shape=1),
)

Check the mean NCC score, mean PC and the standard deviation of the mean PC

In [ ]:
print(xmap_grid_ref.scores.mean())
print(det_grid_ref.pc_average)
print(det_grid_ref.pc_flattened.std(0))

Check the distribution of refined PCs

In [ ]:
det_grid_ref.plot_pc("map")
det_grid_ref.plot_pc("scatter", annotate=True)

The grid is easily recognizable, but the 24th pattern PC can be seen as an outlier.
This can either be set manually, or we can try to find outliers by robustly fitting a line to the (PCz, PCy) plot.
Assuming the sample is perfectly tilted about the detector $X_d$, the slope of the fitted line is the estimated tilt angle.

In [ ]:
xtilt, is_outlier = det_grid_ref.estimate_xtilt(
    detect_outliers=True, degrees=True, return_outliers=True
)
print(xtilt)

We (hopefully) see that 24th pattern is automatically recognized as an outlier.
If not, we can update (or create) the outlier mask manually

In [ ]:
if not is_outlier.any():
    is_outlier[24] = True

We can fit a plane to the remaining PCs.

## Fit a plane to the refined PCs

To fit a plane to the PCs, we must pass an array of all indices in the full map and the indices of the patterns which the PCs were estimated from.
In the case of the grid patterns we've used here, the map indices were returned from the `EBSD.extract_grid()`.

In [ ]:
nav_shape = s.axes_manager.navigation_shape[::-1]
map_indices = np.indices(nav_shape)

is_outlier = is_outlier.reshape(nav_shape_grid)

Using [EBSDDetector.fit_pc()](../reference/generated/kikuchipy.detectors.EBSDDetector.fit_pc.rst) we will fit a plane to the PCs using both an affine transformation and a projective transformation.
The fit method automatically plots the three scatter plots above and a forth 3D plot, with the experimental PC values shown as black circles and a subset of the PCs corresponding to the experimental ones as larger gray circles.

In [ ]:
det_ref_aff = det_grid_ref.fit_pc(
    idx,
    map_indices=map_indices,
    transformation="affine",
    is_outlier=is_outlier,
)
print(det_ref_aff)

# Sample tilt
print(det_ref_aff.sample_tilt)

# Max. deviation between experimental and fitted PC
pc_diff_aff = det_grid_ref.pc - det_ref_aff.pc[tuple(idx)]
print(abs(pc_diff_aff.reshape(-1, 3)).mean(axis=0))

In [ ]:
det_ref_proj = det_grid_ref.fit_pc(
    idx,
    map_indices=map_indices,
    transformation="projective",
    is_outlier=is_outlier,
)
print(det_ref_proj)

# Sample tilt
print(det_ref_proj.sample_tilt)

# Max. deviation between experimental and fitted PC
pc_diff_proj = det_grid_ref.pc - det_ref_proj.pc[tuple(idx)]
print(abs(pc_diff_proj.reshape(-1, 3)).mean(axis=0))

We see that the projective plane better fits the experimental PCs than the affine plane.
But all-in-all, both planes with very well.

## Validate fitted PCs

Finally, we will refine the (already refined) orientations of the grid patterns using the above fitted PCs, and inspect geometrical simulations on top of all patterns.

In [ ]:
det_ref_proj_grid = det_ref_proj.deepcopy()
det_ref_proj_grid.pc = det_ref_proj_grid.pc[tuple(idx)]

In [ ]:
xmap_grid_refori = s_grid.refine_orientation(
    xmap=xmap_grid_ref,
    detector=det_ref_proj_grid,
    master_pattern=mp,
    energy=20,
    method="LN_NELDERMEAD",
    trust_region=[5, 5, 5],
    rtol=1e-5,
    signal_mask=mask,
    chunk_kwargs=dict(chunk_shape=1),
)

print(xmap_grid_refori.scores.mean())

To plot geometrical simulations on top of our experimental patterns, we create a [KikuchiPatternSimulator](../reference/generated/kikuchipy.simulations.KikuchiPatternSimulator.rst) using the Si [Phase](https://orix.readthedocs.io/en/stable/reference/generated/orix.crystal_map.Phase.html) from the master pattern above.
See the [geometrical EBSD simulations tutorial](geometrical_ebsd_simulations.ipynb) for more details.

In [ ]:
rlv = ReciprocalLatticeVector.from_min_dspacing(phase)

rlv.sanitise_phase()  # Expand unit cell

rlv.calculate_structure_factor()

structure_factor = abs(rlv.structure_factor)
rlv = rlv[structure_factor > 0.4 * structure_factor.max()]

rlv.print_table()

In [ ]:
simulator = kp.simulations.KikuchiPatternSimulator(rlv)

Get one simulation per pattern

In [ ]:
sim = simulator.on_detector(
    det_ref_proj_grid,
    xmap_grid_refori.rotations.reshape(*xmap_grid_refori.shape),
)

Plot the geometrical simulations on top of the patterns

In [ ]:
fig, axes = plt.subplots(*grid_shape, figsize=(15, 15))
for i in np.ndindex(grid_shape):
    axes[i].imshow((s_grid.data[i] * ~mask), cmap="gray")
    axes[i].axis("off")

    lines = sim.as_collections(i)[0]
    axes[i].add_collection(lines)

    idx1d = np.ravel_multi_index(i, grid_shape)
    axes[i].text(5, 10, idx1d, c="w", va="top", ha="left", fontsize=20)

fig.subplots_adjust(wspace=0.01, hspace=0.01)

As expected, if we look at the first pattern (0), we see that the patterns shift upwards and to the left as we move down (follow the upper-most near-horizontal Kikuchi line) and to the right (follow the lower left line) in the grid.